In [2]:
from utils_data import StockDataFetcher, generateTargetDataBuySide, generateTargetDataSellSide,getTechnicalIndicators,normalize_dataframe_with_mean_std

In [3]:
fetcher = StockDataFetcher()

stock_symbol = "TSLA"
start_date = "2025-02-25T00:00:00Z"
end_date = "2025-03-02T09:30:00-04:00"

df = fetcher.get_stock_data(stock_symbol, start_date, end_date)
# df

c:\Users\srija\Assignment\Trading\utils_data.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_market_hours.drop(columns=['time'], inplace=True)
c:\Users\srija\Assignment\Trading\utils_data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_market_hours.rename(columns={'time_et': 'time'}, inplace=True)


In [4]:
df = getTechnicalIndicators(df)
df

,close,high,low,trades,open,volume,vwap,time,MA50,RSI,MACD,BB_upper,BB_lower,ADX,CCI,ATR,ROC,OBV
0,313.100,313.7800,312.9101,8115,313.0500,456001,313.271021,2025-02-25 09:59:00-05:00,318.244920,29.349700,-2.245836,321.045435,311.418325,63.167661,-98.274278,1.746396,-1.550168,-5399034
1,311.515,313.2000,311.0000,20581,313.2000,957038,311.709511,2025-02-25 10:00:00-05:00,317.780420,26.877466,-2.345220,320.777912,310.867848,62.931757,-128.432951,1.778796,-1.662037,-6356072
2,311.000,311.7400,310.4500,12553,311.4600,712770,310.741341,2025-02-25 10:01:00-05:00,317.416087,26.137435,-2.437441,320.295853,310.422057,62.855160,-142.196975,1.743882,-1.585393,-7068842
3,311.420,312.6700,310.7410,10361,310.9043,552196,311.829739,2025-02-25 10:02:00-05:00,316.996990,27.814225,-2.448413,319.604013,310.232897,61.824678,-116.467426,1.757105,-1.414675,-6516646
4,310.950,311.5699,310.1000,12722,311.4900,572668,310.708111,2025-02-25 10:03:00-05:00,316.585323,27.101984,-2.466600,319.283227,309.824673,61.081526,-129.257740,1.736590,-0.974807,-7089314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,291.725,292.1800,291.5115,5086,291.6500,337417,291.848434,2025-02-28 15:56:00-05:00,290.986730,55.584289,0.063786,292.105772,289.021738,20.630417,153.435981,1.052665,0.385334,-5893025
1531,292.329,292.3297,291.4700,6071,291.6800,390982,292.021535,2025-02-28 15:57:00-05:00,291.006697,57.637442,0.183024,292.380913,288.967497,20.977128,151.293644,1.038882,0.667723,-5502043
1532,293.110,293.1600,292.1400,8539,292.3200,654559,292.754669,2025-02-28 15:58:00-05:00,291.037697,60.104320,0.336660,292.810814,288.797596,22.022453,186.929632,1.037533,0.933196,-4847484
1533,293.310,293.6200,292.9700,14705,293.1200,1348984,293.269243,2025-02-28 15:59:00-05:00,291.039290,60.710421,0.469149,293.222316,288.682094,23.338075,189.619328,1.009852,1.066451,-3498500


In [30]:
target_buy = generateTargetDataBuySide(df,1.03,0.95)
target_sell = generateTargetDataSellSide(df,0.97,1.05)
# target_buy

Processing Sell Side Data: 100%|██████████| 1535/1535 [00:57<00:00, 26.61it/s] 


In [31]:
len(target_buy[target_buy["action"]=="Target Hit"])

118

In [32]:
len(target_buy[target_buy["action"]=="Stop Loss Hit"])

55

In [33]:
len(target_buy[target_buy["action"]=="End of Day"])

1362

In [34]:
len(target_sell[target_sell["action"]=="Target Hit"])

418

In [35]:
len(target_sell[target_sell["action"]=="Stop Loss Hit"])

15

In [36]:
len(target_sell[target_sell["action"]=="End of Day"])

1102

In [37]:
target_sell[target_sell["action"]=="Target Hit"]["delay"].min()

8.0

In [38]:
target_buy[target_buy["action"]=="Target Hit"]["delay"].min()

9.0

In [16]:
def calculate_optimized_scalping_reward(delay, action_type, success_base_reward=1500, 
                                     failure_base_penalty=1000, min_delay_threshold=3, 
                                     max_reward=2500, decay_rate=0.3,
                                     opportunity_cost_factor=0.2,
                                     missed_opp_multiplier=2.0,
                                     consecutive_successes=0, 
                                     consecutive_success_bonus=0.15):
    """
    Comprehensive reward function optimized for scalping.
    """
    if action_type == 'success':
        # Delay-dependent base reward scaling
        if delay <= min_delay_threshold:
            base_reward = max_reward - (max_reward - success_base_reward) * (delay / min_delay_threshold)
        else:
            base_reward = success_base_reward
        
        # Apply exponential decay
        reward = base_reward * np.exp(-decay_rate * delay)
        
        # Apply opportunity cost
        opportunity_cost = opportunity_cost_factor * delay * success_base_reward
        opportunity_cost = min(opportunity_cost, reward * 0.8)
        reward = reward - opportunity_cost
        
        # Apply sequential bonus
        if consecutive_successes > 0:
            sequential_bonus = reward * (consecutive_success_bonus * consecutive_successes)
            reward += sequential_bonus
        
        return reward
    
    elif action_type == 'failure':
        # Standard penalty with exponential decay
        penalty = -failure_base_penalty * np.exp(-decay_rate * delay)
        
        # Add opportunity cost to penalty
        opportunity_cost = opportunity_cost_factor * delay * failure_base_penalty
        penalty = penalty - opportunity_cost
        
        return penalty
    
    elif action_type == 'missed_opportunity':
        # Enhanced penalty for missed opportunities
        missed_penalty = -failure_base_penalty * missed_opp_multiplier * np.exp(-decay_rate * delay)
        return missed_penalty
    
    elif action_type == 'no_action':
        # Reward for correctly staying out of the market
        return 100
